# 1. Build docker image

In [1]:
!docker build -t pytorch-bert-base-uncased -f Dockerfile .

Sending build context to Docker daemon  951.3kB
Step 1/6 : FROM public.ecr.aws/bitnami/pytorch:latest
latest: Pulling from bitnami/pytorch

435fd20e: Pulling fs layer 
c2529f5d: Pulling fs layer 
40739f4c: Pulling fs layer 
ac48cf6b: Pulling fs layer 
2f619d46: Pulling fs layer 
4dcfee87: Pulling fs layer 
1b0c50f4: Pulling fs layer 
473f00af: Pulling fs layer 
0d6fc61e: Pull complete 4.7MB/564.7MBBDownloading  440.7MB/564.7MBDigest: sha256:9bc7b87b81fb1d5fe396261caecf6b95b61ff3cbbd4d4095ca197597f9769fe6
Status: Downloaded newer image for public.ecr.aws/bitnami/pytorch:latest
 ---> 47335f066f16
Step 2/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Running in 06e655fb0afc
Removing intermediate container 06e655fb0afc
 ---> c884da36886b
Step 3/6 : COPY ./code /opt/ml/code
 ---> 1a3a4c7ed0e9
Step 4/6 : RUN ls -l /opt/ml/code &&     pip install -r /opt/ml/code/requirements.txt
 ---> Running in 3231cbfc064b
total 32
drwxrwxr-x 2 root root  4096 Nov 30 03:59 __pycache__
-rw-rw-r-- 1 root root  322

# 2. Push to ECR

In [2]:
import boto3

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = "pytorch-bert-base-uncased"

ecr_image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

print(ecr_image)

833719260605.dkr.ecr.us-east-1.amazonaws.com/pytorch-bert-base-uncased:latest


In [ ]:
# !aws ecr get-login-password | docker login xxxx -U AWS --password-stdin
# !docker tag pytorch-bert-base-uncased:latest ECR_IMAGE 
# !docker push ECR_IMAGE

In [3]:
%%sh

# The name of our algorithm
algorithm_name=pytorch-bert-base-uncased

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# # Get the login command from ECR in order to pull down the SageMaker PyTorch image
# $(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# # Build the docker image locally with the image name and then push it to ECR
# # with the full name.

# docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
The push refers to repository [833719260605.dkr.ecr.us-east-1.amazonaws.com/pytorch-bert-base-uncased]
5e1f5763557f: Preparing
afd64b4c3656: Preparing
8de3ac157a9f: Preparing
cb4d32b6b8f3: Preparing
761ec9808e93: Preparing
30fa00a9fc02: Preparing
b7c8cba4cc48: Preparing
33c1b39847d7: Preparing
61f63cc748f7: Preparing
bacd2038ff01: Preparing
5b2e78f7eb38: Preparing
33c1b39847d7: Waiting
61f63cc748f7: Waiting
bacd2038ff01: Waiting
5b2e78f7eb38: Waiting
30fa00a9fc02: Waiting
b7c8cba4cc48: Waiting
afd64b4c3656: Pushed
761ec9808e93: Pushed
cb4d32b6b8f3: Pushed
30fa00a9fc02: Pushed
61f63cc748f7: Pushed
5e1f5763557f: Pushed
bacd2038ff01: Pushed
5b2e78f7eb38: Pushed
33c1b39847d7: Pushed
8de3ac157a9f: Pushed
b7c8cba4cc48: Pushed
latest: digest: sha256:8483606034fe3486e88ce1184ac6209ad3c593a4309771d1836d0d0fa602dfaa size: 2631


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



# 3. Deploy to Sagemaker Endpoint

In [34]:
from sagemaker.estimator import Estimator
# help(Estimator.fit)
help(Estimator)

Help on class Estimator in module sagemaker.estimator:

class Estimator(EstimatorBase)
 |  A generic Estimator to train using any supplied algorithm.
 |  
 |  This class is designed for use with algorithms that don't have their own, custom class.
 |  
 |  Method resolution order:
 |      Estimator
 |      EstimatorBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, image_uri, role, instance_count=None, instance_type=None, volume_size=30, volume_kms_key=None, max_run=86400, input_mode='File', output_path=None, output_kms_key=None, base_job_name=None, sagemaker_session=None, hyperparameters=None, tags=None, subnets=None, security_group_ids=None, model_uri=None, model_channel_name='model', metric_definitions=None, encrypt_inter_container_traffic=False, use_spot_instances=False, max_wait=None, checkpoint_s3_uri=None, checkpoint_local_path=None, enable_network_isolation=False, rules=None, debugger_hook_config=None, tensorboard_output_config=None, enable_sagem

In [6]:
import sagemaker
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::833719260605:role/ivy-sagemaker-role


## 3.1 Put the pretrained model in S3

In [11]:
!pip install transformers==2.3.0

     |████████████████████████████████| 447 kB 23.7 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 70.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 67.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [15]:

my_model_name="bert-base-uncased"
# from transformers import pipeline
# bert_model = pipeline("bert-base-uncased")
# bert_model.save_pretrained("./model")

from transformers import BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(my_model_name)
bert_model.save_pretrained("./model")

In [16]:
# from transformers import BertForSequenceClassification
# help(BertForSequenceClassification)

In [17]:
!cd model && tar czvf ../model.tar.gz *
!cd ../ && ls -l
!pwd

config.json
pytorch_model.bin
total 32
drwxrwxr-x 6 ec2-user ec2-user  4096 Nov 30 04:32 amazon-sagemaker-bert-pytorch
drwxrwxr-x 4 ec2-user ec2-user  4096 Nov 29 08:42 docker_test_folder
drwx------ 2 root     root     16384 Nov 29 03:17 lost+found
drwxrwxr-x 6 ec2-user ec2-user  4096 Nov 29 08:41 sagemaker_train_demo
-rw-rw-r-- 1 ec2-user ec2-user  1140 Nov 30 04:00 Untitled.ipynb


In [19]:
!cd /home/ec2-user/SageMaker/amazon-sagemaker-bert-pytorch && ls -l

total 396112
-rw-rw-r-- 1 ec2-user ec2-user     22068 Nov 30 04:13 bert-sm-python-SDK.ipynb
drwxrwxr-x 4 ec2-user ec2-user      4096 Nov 30 04:18 code
-rw-rw-r-- 1 ec2-user ec2-user       309 Nov 30 03:59 CODE_OF_CONDUCT.md
-rw-rw-r-- 1 ec2-user ec2-user    255330 Nov 30 03:59 cola_public_1.1.zip
-rw-rw-r-- 1 ec2-user ec2-user      3306 Nov 30 03:59 CONTRIBUTING.md
-rw-rw-r-- 1 ec2-user ec2-user      1660 Nov 30 03:59 Dockerfile
-rw-rw-r-- 1 ec2-user ec2-user       927 Nov 30 03:59 LICENSE
drwxrwxr-x 2 ec2-user ec2-user      4096 Nov 30 04:31 model
-rw-rw-r-- 1 ec2-user ec2-user 405254398 Nov 30 04:33 model.tar.gz
-rw-rw-r-- 1 ec2-user ec2-user     43678 Nov 30 04:33 pytorch-docker-deploy.ipynb
-rw-rw-r-- 1 ec2-user ec2-user      4957 Nov 30 03:59 README.md


In [20]:
import os
import boto3
import sagemaker

bucket = sess.default_bucket()
prefix = "sagemaker/pytorch-bert-base-uncased"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

model_s3_link = "s3://{}".format(os.path.join(bucket, key))
print(model_s3_link)

s3://sagemaker-us-east-1-833719260605/sagemaker/pytorch-bert-base-uncased/model.tar.gz


In [21]:
from sagemaker.estimator import Estimator

# hyperparameters = {"epochs": 1}
hyperparameters={
        "epochs": 1,
        "num_labels": 2,
        "backend": "gloo",
        "lr":0.1
    }

instance_type = "ml.t2.medium" #"ml.m4.xlarge"
# model_uri="s3://sagemaker-us-east-1-420737321821/sagemaker/ivy-demo-pytorch-bert/pytorch-training-2021-11-29-14-48-31-535/output/model.tar.gz"
model_uri=model_s3_link
estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    image_uri=ecr_image,
    model_uri=model_uri,
    hyperparameters=hyperparameters,
)

estimator.fit()

predictor = estimator.deploy(1, instance_type)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'ml.t2.medium' at 'resourceConfig.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.p4d.24xlarge, ml.c5n.xlarge, ml.p3.16xlarge, ml.m5.large, ml.p2.16xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.c5.4xlarge, ml.c5n.18xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c4.8xlarge, ml.g4dn.2xlarge, ml.c5.9xlarge, ml.g4dn.4xlarge, ml.c5.xlarge, ml.g4dn.16xlarge, ml.c4.xlarge, ml.g4dn.8xlarge, ml.c5n.2xlarge, ml.c5n.4xlarge, ml.c5.18xlarge, ml.p3dn.24xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, ml.c5n.9xlarge, ml.m5.12xlarge, ml.m4.xlarge, ml.m5.24xlarge, ml.m4.2xlarge, ml.p2.8xlarge, ml.m5.2xlarge, ml.p3.8xlarge, ml.m4.4xlarge]

In [9]:
# import boto3
# from sagemaker.amazon.amazon_estimator import get_image_uri
# training_image = get_image_uri(boto3.Session().region_name, 'image-classification')
# print(training_image)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [22]:
print(ecr_image)

833719260605.dkr.ecr.us-east-1.amazonaws.com/pytorch-bert-base-uncased:latest


In [23]:
!docker run pytorch-bert-base-uncased:latest

 04:44:54.96 
 04:44:54.97 Welcome to the Bitnami pytorch container
 04:44:54.98 Subscribe to project updates by watching https://github.com/bitnami/bitnami-docker-pytorch
 04:44:54.98 Submit issues and feature requests at https://github.com/bitnami/bitnami-docker-pytorch/issues
 04:44:54.98 



In [25]:
!docker ps -a

CONTAINER ID   IMAGE                              COMMAND                  CREATED          STATUS                      PORTS     NAMES
e7ab6cd208d1   pytorch-bert-base-uncased:latest   "/opt/bitnami/script…"   20 seconds ago   Exited (0) 18 seconds ago             sad_goldstine


In [31]:
!docker logs sad_goldstine

 04:44:54.96 
 04:44:54.97 Welcome to the Bitnami pytorch container
 04:44:54.98 Subscribe to project updates by watching https://github.com/bitnami/bitnami-docker-pytorch
 04:44:54.98 Submit issues and feature requests at https://github.com/bitnami/bitnami-docker-pytorch/issues
 04:44:54.98 



In [33]:
# !docker help

# 4. Test the endpoint

In [35]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   bert-sm-python-SDK.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/Dockerfile-checkpoint
	.ipynb_checkpoints/pytorch-docker-deploy-checkpoint.ipynb
	code/.ipynb_checkpoints/
	model.tar.gz

no changes added to commit (use "git add" and/or "git commit -a")


In [36]:
!git push

Username for 'https://github.com/songm28/amazon-sagemaker-bert-pytorch.git': ^C


In [37]:
!git -h

unknown option: -h
usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]
